In [ ]:
import os
import re
import imaplib

from dotenv import load_dotenv
from datetime import datetime
import pandas as pd

In [ ]:
from main import (
    get_transactions_from_imap, 
    save_transactions_to_file, 
    export_transactions_text, 
    remove_duplicates_from_list,
    export_transactions_df
)

In [ ]:
load_dotenv()

IMAP_USER_EMAIL_ADDRESS = os.getenv('IMAP_USER_EMAIL_ADDRESS')
IMAP_USER_PASSWORD = os.getenv('IMAP_USER_PASSWORD')
IMAP_SERVER = os.getenv('IMAP_SERVER')
IMAP_FILTER=os.getenv('IMAP_FILTER')

# Fetch the environment variable 'IMAP_BANK_MSG_STARTS' and split it by |.
# to get a list of start strings for bank messages. This is done to handle
# multiple possible start strings for different types of bank messages.
IMAP_BANK_MSG_STARTS = os.getenv('IMAP_BANK_MSG_STARTS').split('|')

IMAP_BANK_MSG_END=os.getenv('IMAP_BANK_MSG_END')
REGEX_BANK_FILE=os.getenv('REGEX_BANK_FILE')

In [ ]:
# this is done to make SSL connection with GMAIL
imap_con = imaplib.IMAP4_SSL(IMAP_SERVER)

# logging the user in
imap_con.login(IMAP_USER_EMAIL_ADDRESS, IMAP_USER_PASSWORD)

# calling function to check for email under this label
imap_con.select('Inbox')

(transactions_text, email_with_problems, emails_without_transactions) = get_transactions_from_imap(imap_con)

In [ ]:
transactions_text_without_duplicates = remove_duplicates_from_list(transactions_text)

In [ ]:
notification_regex = []
with open(REGEX_BANK_FILE) as regex_bank_file:
    for line in regex_bank_file:
        notification_regex.append(line.strip('\n'))

now = datetime.now()
dt_string = now.strftime("%Y-%m-%d_%H:%M:%S")
csv_outputfile = f"target/transactions-{dt_string}.csv"
text_outputfile = f"target/transactions-{dt_string}.text"
save_transactions_to_file(transactions_text_without_duplicates, text_outputfile)
export_transactions_text(
    transactions_text=transactions_text_without_duplicates, 
    transaction_regex=notification_regex, 
    format="csv",
    outputfile=csv_outputfile
)

df_transactions=export_transactions_df(transactions_text_without_duplicates, notification_regex)

In [ ]:
df_transactions

In [ ]:
import pandas as pd
import re

# Load CSVs
df_accounts = pd.read_csv("accounts.csv", dtype={"DestAccount": str})
df_trips = pd.read_csv("trips.csv")
df_provider_name = pd.read_csv("ProviderName.csv")

# Function to determine trip name
def get_trip_name(date):
    for idx, row in df_trips.iterrows():
        if row['StartDate'] <= date <= row['EndDate']:
            return row['TripName']
    return None

# Function to determine trip spend type
def get_trip_spendtype(date):
    for idx, row in df_trips.iterrows():
        if row['StartDate'] <= date <= row['EndDate']:
            return row['SpendType']
    return None

# Function to clean and standardize text
def clean_text(text):
    clean = text.lower()  # Convert to lowercase
    clean = re.sub(r'[^a-z0-9\s]', '', clean)  # Remove special characters
    clean = re.sub(r'\s+', ' ', clean).strip()  # Replace multiple spaces with a single space and trim
    return clean

# Apply the cleaning function to 'dest' column and store in 'DestClean'
df_transactions['DestClean'] = df_transactions['dest'].apply(clean_text)

# Clean the provider names as well for consistent matching
df_provider_name['ProviderDestNameClean'] = df_provider_name['ProviderDestName'].apply(clean_text)

# Function to apply provider rules using the cleaned data
def apply_provider_rules_clean(row):
    if pd.isna(row['SpendType']):
        for idx, provider_rule in df_provider_name.iterrows():
            if provider_rule['Filter'] == 'contains' and provider_rule['ProviderDestNameClean'] in row['DestClean']:
                row['SpendType'] = provider_rule['SpendType']
                row['ProviderGroup'] = provider_rule['ProviderGroup']
                return row
    return row

# Merge transactions with accounts based on 'DestClean'
df_merged = df_transactions.merge(df_accounts, left_on='DestClean', right_on='DestAccount', how='left')

# Convert date columns to datetime
df_merged['DateTime'] = pd.to_datetime(df_merged['DateTime'])
df_trips['StartDate'] = pd.to_datetime(df_trips['StartDate'])
df_trips['EndDate'] = pd.to_datetime(df_trips['EndDate'])

# Add TripName and SpendType based on the transaction date
df_merged['TripName'] = df_merged['DateTime'].apply(get_trip_name)
df_merged['SpendType'] = df_merged['DateTime'].apply(get_trip_spendtype).combine_first(df_merged['SpendType'])

# Assign 'withdraw' to SpendType for transactions with type 'retiro'
df_merged.loc[df_merged['Type'] == 'Retiro', 'SpendType'] = 'withdraw'

# Create ProviderGroup column
df_merged['ProviderGroup'] = None

# Apply provider rules using the cleaned data
df_merged = df_merged.apply(apply_provider_rules_clean, axis=1)


In [ ]:
# Filter out transactions that are not categorized
uncategorized = df_merged[df_merged['SpendType'].isna()]

# Group by month and sum the amounts
monthly_uncategorized = uncategorized.groupby(uncategorized['DateTime'].dt.strftime('%Y-%m'))['ValueFloat'].sum().reset_index()
monthly_uncategorized.columns = ['Month', 'TotalUncategorized']

# Calculate the total amount not categorized
total_uncategorized = monthly_uncategorized['TotalUncategorized'].sum()

# Convert the total amount and monthly amounts to millions
total_uncategorized_millions = total_uncategorized / 1_000_000
monthly_uncategorized['TotalUncategorized'] = monthly_uncategorized['TotalUncategorized'] / 1_000_000

print(f"Total no categorizado: {total_uncategorized_millions:.2f} millones de pesos")

# Filter out transactions that have not been categorized
uncategorized = df_merged[df_merged['SpendType'].isna()]

# Get the top 10 most frequent uncategorized transactions based on ValueFloat
most_frequent_by_value = uncategorized.groupby('DestClean').agg(Count=('DestClean', 'size'), TotalValue=('ValueFloat', 'sum')).nlargest(50, 'TotalValue')

print("Top 50 transacciones no categorizadas basadas en el valor total:")
print(most_frequent_by_value)


In [ ]:
df_merged

In [ ]:
import matplotlib.pyplot as plt

# Group by SpendType and sum the ValueFloat
spendtype_summary = df_merged.groupby('SpendType')['ValueFloat'].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(15,10))
spendtype_summary.plot(kind='bar', color='skyblue')
plt.title('Total Expenditure by SpendType')
plt.ylabel('Total Expenditure')
plt.xlabel('SpendType')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Grouping the data by month and SpendType and summing the amounts
monthly_spendtype_data = df_merged.groupby([df_merged['DateTime'].dt.strftime('%Y-%m'), 'SpendType'])['ValueFloat'].sum().reset_index()

# Reshaping the data for heatmap
heatmap_data = monthly_spendtype_data.pivot(index='DateTime', columns='SpendType', values='ValueFloat')
heatmap_data = heatmap_data.fillna(0)  # Replacing NaN values with 0 for the heatmap

# Plotting the heatmap
plt.figure(figsize=(15, 8))
sns.heatmap(heatmap_data, cmap='YlGnBu', annot=True, fmt=".0f", linewidths=.5)
plt.title('Monthly Spending by Spend Type')
plt.ylabel('Month')
plt.xlabel('Type of Spend')
plt.tight_layout()

plt.show()


In [ ]:
# Adjusting the data to ensure alignment of withdrawals with expenses and income
# Grouping the data by month for 2022 and summing the expenses
data_2022 = df_merged[df_merged['DateTime'].dt.year == 2022].groupby(df_merged['DateTime'].dt.strftime('%Y-%m')).agg(TotalExpenses=('ValueFloat', 'sum')).reset_index()

# Grouping the data by month for 2022 and summing the withdrawals
withdrawals_2022 = df_merged[(df_merged['DateTime'].dt.year == 2022) & (df_merged['Type'] == 'Retiro')].groupby(df_merged['DateTime'].dt.strftime('%Y-%m')).agg(TotalWithdrawals=('ValueFloat', 'sum')).reset_index()

# Merging the two datasets on month-year
data_2022 = data_2022.merge(withdrawals_2022, on='DateTime', how='left')
data_2022['TotalWithdrawals'].fillna(0, inplace=True)  # Replacing NaN values with 0
data_2022['Income'] = 7500000  # Your monthly income

# Plotting the monthly comparison
plt.figure(figsize=(15, 7))
sns.barplot(data=data_2022, x='DateTime', y='Income', color='lightgreen', label='Ingresos')
sns.barplot(data=data_2022, x='DateTime', y='TotalExpenses', color='skyblue', label='Gastos Totales')
sns.barplot(data=data_2022, x='DateTime', y='TotalWithdrawals', bottom=data_2022['TotalExpenses'], color='salmon', label='Retiros Totales')
plt.title('Comparación de Salario vs. Gastos y Retiros por Mes (2022)')
plt.ylabel('Cantidad (en millones de pesos)')
plt.xlabel('Mes (2022)')
plt.legend()
plt.tight_layout()

plt.show()


In [ ]:
# Filtering data for transactions related to trips
trip_data = df_merged[~df_merged['TripName'].isna()]

# Plotting the trip expenses over time
plt.figure(figsize=(15, 7))
sns.lineplot(data=trip_data, x='DateTime', y='ValueFloat', hue='TripName', marker='o')
plt.title('Gastos Relacionados con Viajes a lo Largo del Tiempo')
plt.ylabel('Cantidad (en pesos)')
plt.xlabel('Fecha')
plt.legend(title='Nombre del Viaje')
plt.tight_layout()

plt.show()


In [ ]:
# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.barplot(data=monthly_uncategorized, x='Month', y='TotalUncategorized', palette="Blues_d")
plt.title('Monto no categorizado por mes (en millones de pesos)')
plt.ylabel('Monto (en millones)')
plt.xlabel('Mes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
text_outputfile = f"target/transactions-final-{dt_string}.csv"
df_merged.to_csv(text_outputfile, index=False)